In [0]:
import torch, torchvision
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
  
import numpy as np
import glob
import shutil

In [0]:
!wget http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz
!tar -xvzf 101_ObjectCategories.tar.gz

In [0]:
import os
os.makedirs("101_ObjectCategories/train")
os.makedirs("101_ObjectCategories/valid")

In [0]:
def get_immediate_subdirectories(data_directory):
    return [name for name in os.listdir(data_directory)
            if os.path.isdir(os.path.join(data_directory, name))]

def move_images(directory, train_dir, valid_dir, image_list, label):
  # shuffle all images
  random_set = np.random.permutation(len(image_list))
  
  # select 80% of images at random for train
  train_list = random_set[:round(len(random_set) * 0.8)]
  
  # select 20% of images at randomf or valid
  valid_list = random_set[-(len(image_list) - len(train_list))::]
  
  train_images = []
  valid_images = []
  
  for index in train_list:
    train_images.append(image_list[index])
  
  for index in valid_list:
    valid_images.append(image_list[index])
  
  for train_image in train_images:
    os.rename(os.path.join(directory, label, train_image), os.path.join(train_dir, label, train_image))
  for valid_image in valid_images:
    os.rename(os.path.join(directory, label, valid_image), os.path.join(valid_dir, label, valid_image))
  
  shutil.rmtree(os.path.join(directory, label))
    
#   os.removedirs(os.path.join(directory, label))

In [0]:
# !rm -r 101_ObjectCategories/
# !tar -xvzf 101_ObjectCategories.tar.gz
# reference: https://gist.github.com/GertjanBrouwer/95c815565d3d8788137929ef27054db9

train_dir = "101_ObjectCategories/train/"
valid_dir = "101_ObjectCategories/valid/"

dir_list = get_immediate_subdirectories("101_ObjectCategories/")
directory = "101_ObjectCategories/"

# os.mkdir(train_dir)
# os.mkdir(valid_dir)

for dir_ in dir_list:
  if dir_ == "train" or dir_ == "valid":
    continue
    
  label = dir_
  
  print('Started moving: ' + str(label))
  
  image_list = [os.path.basename(x) for x in glob.glob(os.path.join(directory, label) + '/*.jpg')]
  
  if not os.path.exists(os.path.join(train_dir, label)):
    os.mkdir(os.path.join(train_dir, label))
  
  if not os.path.exists(os.path.join(valid_dir, label)):
    os.mkdir(os.path.join(valid_dir, label))
  
  directory = "101_ObjectCategories/"
  
  move_images(directory, train_dir, valid_dir, image_list, label)
  
  print("Finished moving: " + str(label))

Started moving: elephant
Finished moving: elephant
Started moving: watch
Finished moving: watch
Started moving: euphonium
Finished moving: euphonium
Started moving: bonsai
Finished moving: bonsai
Started moving: flamingo
Finished moving: flamingo
Started moving: crocodile_head
Finished moving: crocodile_head
Started moving: sea_horse
Finished moving: sea_horse
Started moving: crayfish
Finished moving: crayfish
Started moving: chandelier
Finished moving: chandelier
Started moving: panda
Finished moving: panda
Started moving: ceiling_fan
Finished moving: ceiling_fan
Started moving: schooner
Finished moving: schooner
Started moving: headphone
Finished moving: headphone
Started moving: pyramid
Finished moving: pyramid
Started moving: minaret
Finished moving: minaret
Started moving: platypus
Finished moving: platypus
Started moving: dalmatian
Finished moving: dalmatian
Started moving: strawberry
Finished moving: strawberry
Started moving: cellphone
Finished moving: cellphone
Started moving:

In [0]:
from torchvision import transforms

# Image transformations
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    'valid':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [0]:
from torchvision import datasets
from torch.utils.data import DataLoader

traindir = '101_ObjectCategories/train'
validdir = '101_ObjectCategories/valid'

batch_size = 64

# Datasets from folders
data = {
    'train':
    datasets.ImageFolder(root=traindir, transform=image_transforms['train']),
    'valid':
    datasets.ImageFolder(root=validdir, transform=image_transforms['valid']),
}

# Dataloader iterators, make sure to shuffle
dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True),
    'val': DataLoader(data['valid'], batch_size=batch_size, shuffle=True)
}

In [0]:
trainiter = iter(dataloaders['train'])
features, labels = next(trainiter)
features.shape, labels.shape


(torch.Size([64, 3, 224, 224]), torch.Size([64]))

In [0]:
from torchvision import models
model = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
102502400it [00:02, 46613726.04it/s]


In [0]:
# Freeze model weights
for param in model.parameters():
    param.requires_grad = False

In [0]:
save_file_name = 'resnet50-transfer-4.pt'
checkpoint_path = 'resnet50-transfer-4.pth'

In [0]:
# import torch.nn as nn

n_inputs = model.fc.in_features

# Source: Abhijeet Patil, IIT B
# model.fc = nn.Linear(n_inputs, 102)

# Add on classifier
model.fc = nn.Sequential(
                      nn.Linear(n_inputs, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(256, 100),                   
                      nn.LogSoftmax(dim=1))

In [0]:
# Find total parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

24,058,276 total parameters.
550,244 training parameters.


In [0]:
import torch
t = torch.Tensor([1, 2, 3])
t.to(device)

# model = model.to('cuda')
# Distribute across 2 gpus
# model = nn.DataParallel(model)

RuntimeError: ignored

In [0]:
# from torch import optim
# Loss and optimizer
criterion = nn.NLLLoss()

# Source: Abhijeet Patil, IIT B
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9) 

optimizer = optim.Adam(model.parameters())

In [0]:
def train(model,
          criterion,
          optimizer,
          train_loader,
          valid_loader,
          save_file_name,
          max_epochs_stop=3,
          n_epochs=20,
          print_every=2):
    """Train a PyTorch Model

    Params
    --------
        model (PyTorch model): cnn to train
        criterion (PyTorch loss): objective to minimize
        optimizer (PyTorch optimizier): optimizer to compute gradients of model parameters
        train_loader (PyTorch dataloader): training dataloader to iterate through
        valid_loader (PyTorch dataloader): validation dataloader used for early stopping
        save_file_name (str ending in '.pt'): file path to save the model state dict
        max_epochs_stop (int): maximum number of epochs with no improvement in validation loss for early stopping
        n_epochs (int): maximum number of training epochs
        print_every (int): frequency of epochs to print training stats

    Returns
    --------
        model (PyTorch model): trained cnn with best weights
        history (DataFrame): history of train and validation loss and accuracy
    """
    train_on_gpu = True
    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = time.time()

    # Main loop
    for epoch in range(n_epochs):

        # keep track of training and validation loss each epoch
        train_loss = 0.0
        valid_loss = 0.0

        train_acc = 0
        valid_acc = 0

        # Set to training
        model.train()
        start = time.time()

        # Training loop
        for ii, (data, target) in enumerate(train_loader):
            # Tensors to gpu
            if train_on_gpu:
                print(data)
#                 data, target = data.cuda(), target.cuda()
                data = data.to(device)
                target = target.to(device)

            # Clear gradients
            optimizer.zero_grad()
            # Predicted outputs are log probabilities
            output = model(data)

            # Loss and backpropagation of gradients
            loss = criterion(output, target)
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Track train loss by multiplying average loss by number of examples in batch
            train_loss += loss.item() * data.size(0)

            # Calculate accuracy by finding max log probability
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            # Need to convert correct tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            # Multiply average accuracy times the number of examples in batch
            train_acc += accuracy.item() * data.size(0)

            # Track training progress
            print(
                f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. {time.time() - start:.2f} seconds elapsed in epoch.',
                end='\r')

        # After training loops ends, start validation
        else:
            model.epochs += 1

            # Don't need to keep track of gradients
            with torch.no_grad():
                # Set to evaluation mode
                model.eval()

                # Validation loop
                for data, target in valid_loader:
                    # Tensors to gpu
                    if train_on_gpu:
                        print(data)
#                         data, target = data.cuda(), target.cuda()
                        data = data.to(device)
                        target = target.to(device)

                    # Forward pass
                    output = model(data)

                    # Validation loss
                    loss = criterion(output, target)
                    # Multiply average loss times the number of examples in batch
                    valid_loss += loss.item() * data.size(0)

                    # Calculate validation accuracy
                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
                    # Multiply average accuracy times the number of examples
                    valid_acc += accuracy.item() * data.size(0)

                # Calculate average losses
                train_loss = train_loss / len(train_loader.dataset)
                valid_loss = valid_loss / len(valid_loader.dataset)

                # Calculate average accuracy
                train_acc = train_acc / len(train_loader.dataset)
                valid_acc = valid_acc / len(valid_loader.dataset)

                history.append([train_loss, valid_loss, train_acc, valid_acc])

                # Print training and validation results
                if (epoch + 1) % print_every == 0:
                    print(
                        f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}'
                    )
                    print(
                        f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%'
                    )

                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min:
                    # Save model
                    torch.save(model.state_dict(), save_file_name)
                    # Track improvement
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                    valid_best_acc = valid_acc
                    best_epoch = epoch

                # Otherwise increment count of epochs with no improvement
                else:
                    epochs_no_improve += 1
                    # Trigger early stopping
                    if epochs_no_improve >= max_epochs_stop:
                        print(
                            f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
                        )
                        total_time = time.time() - overall_start
                        print(
                            f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
                        )

                        # Load the best state dict
                        model.load_state_dict(torch.load(save_file_name))
                        # Attach the optimizer
                        model.optimizer = optimizer

                        # Format history
                        history = pd.DataFrame(
                            history,
                            columns=[
                                'train_loss', 'valid_loss', 'train_acc',
                                'valid_acc'
                            ])
                        return model, history

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = time.time() - overall_start
    print(
        f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
    )
    print(
        f'{total_time:.2f} total seconds elapsed. {total_time / (epoch):.2f} seconds per epoch.'
    )
    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])
    return model, history

In [0]:
import time
import pandas as pd
model, history = train(
    model,
    criterion,
    optimizer,
    dataloaders['train'],
    dataloaders['val'],
    save_file_name=save_file_name,
    max_epochs_stop=5,
    n_epochs=30,
    print_every=2)

Starting Training from Scratch.

tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044

NameError: ignored

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [0]:
import time
import pandas as pd

model, history = train(
    model,
    criterion,
    optimizer,
    dataloaders['train'],
    dataloaders['val'],
    save_file_name=save_file_name,
    max_epochs_stop=5,
    n_epochs=30,
    print_every=2)

Model has been trained for: 0 epochs.



RuntimeError: ignored